<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/TSP_Heuristiken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TSP - Heuristiken

## Daten

In [1]:
!pip install haversine
!pip install pandas
!pip install plotly.express

import haversine
import pandas as pd
import plotly.express as px

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
# Daten erstellen
input_data = dict()
input_data["Halle (Saale)"]=dict(id=1, lon=11.9672201241301, lat=51.4970124613279, pop=238.061)
input_data["Magdeburg"]=dict(id=2, lon=11.6363317198121, lat=52.1229002894915, pop=236.188)
input_data["Dessau-Roßlau"]=dict(id=3, lon=12.2331141970666, lat=51.8442730013004, pop=78.731)
input_data["Lutherstadt Wittenberg"]=dict(id=4, lon=12.6357720180488, lat=51.8737704942318, pop=44.984)
input_data["Weißenfels"]=dict(id=5, lon=11.9683230903702, lat=51.1983573259, pop=39.745)
input_data["Halberstadt"]=dict(id=6, lon=11.0495250643003, lat=51.8912329116255, pop=38.682)
input_data["Stendal"]=dict(id=7, lon=11.8508504963119, lat=52.602278283988, pop=38.359)

# Daten als Dataframe
df_input = {col_name: [input_data[name][col_name] for name in input_data] for col_name in ['id', 'lon', 'lat', 'pop']}
df_input['name'] = input_data.keys()
df_input = pd.DataFrame(data=df_input, columns=['name', 'id', 'pop', 'lon', 'lat'])

# print(df)
print(df_input.to_latex(index=False))

# Distanzen berechnen
dist = {i: {j: int(haversine.haversine((input_data[i]['lat'], input_data[i]['lon']), (input_data[j]['lat'], input_data[j]['lon'])))
            for j in input_data}
        for i in input_data}

# Distanzen als Dataframe
dist_df = {input_data[destin]['id']: [dist[origin][destin] for origin in input_data.keys()] for destin in input_data.keys()}
dist_df['origin'] = input_data.keys()
dist_df['id'] = [input_data[n]['id'] for n in input_data.keys()]
dist_df = pd.DataFrame(data=dist_df,
                       columns=['id', 'origin'] + [i+1 for i in range(len(input_data.keys()))]
                       )

# print(dist_df)
print(dist_df.to_latex(index=False))


\begin{tabular}{lrrrr}
\toprule
                  name &  id &     pop &       lon &       lat \\
\midrule
         Halle (Saale) &   1 & 238.061 & 11.967220 & 51.497012 \\
             Magdeburg &   2 & 236.188 & 11.636332 & 52.122900 \\
         Dessau-Roßlau &   3 &  78.731 & 12.233114 & 51.844273 \\
Lutherstadt Wittenberg &   4 &  44.984 & 12.635772 & 51.873770 \\
            Weißenfels &   5 &  39.745 & 11.968323 & 51.198357 \\
           Halberstadt &   6 &  38.682 & 11.049525 & 51.891233 \\
               Stendal &   7 &  38.359 & 11.850850 & 52.602278 \\
\bottomrule
\end{tabular}

\begin{tabular}{rlrrrrrrr}
\toprule
 id &                 origin &   1 &   2 &  3 &   4 &   5 &   6 &   7 \\
\midrule
  1 &          Halle (Saale) &   0 &  73 & 42 &  62 &  33 &  76 & 123 \\
  2 &              Magdeburg &  73 &   0 & 51 &  73 & 105 &  47 &  55 \\
  3 &          Dessau-Roßlau &  42 &  51 &  0 &  27 &  74 &  81 &  88 \\
  4 & Lutherstadt Wittenberg &  62 &  73 & 27 &   0 &  88 & 108 &  

<ipython-input-23-b9dd64fdc77b>:17: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.

<ipython-input-23-b9dd64fdc77b>:33: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



## Nächster Nachbar Heuristik

In [40]:
tour_info = {'heuristik': [], 'start': [], 'distance': [], 'tour': []}

# for heuristik in ['Nächster Nachbar']:
# for heuristik in ['Cheapest Insertion']:
for heuristik in ['Nächster Nachbar', 'Cheapest Insertion']:

  if heuristik == 'Nächster Nachbar':
    print('')
    print('NÄCHSTER NACHBAR')
    for depot in input_data.keys():
      tour = [depot]
      total_distance = 0
      while len(tour) < len(input_data.keys()):
        next_node = None
        for destin in input_data.keys():
          if destin not in tour:
            if next_node is None or dist[tour[-1]][next_node] > dist[tour[-1]][destin]:
              next_node = destin

        total_distance += dist[tour[-1]][next_node]
        tour.append(next_node)

      total_distance += dist[tour[-1]][depot]
      tour.append(depot)

      # Info für Ergebnis-Tabelle
      tour_info['heuristik'].append(heuristik)
      tour_info['start'].append(depot)
      tour_info['distance'].append(total_distance)
      tour_info['tour'].append(tour)

      print('Startknoten: ' + str(depot) + r" \\")
      print('  Tour: ' + str(tour) + r" \\")
      print('  Distance: ' + str(total_distance) + ' = sum(' + str([dist[tour[n]][tour[n+1]] for n in range(len(tour)-1)]) + r") \\~\\")


  elif heuristik == 'Cheapest Insertion':
    print('')
    print('CHEAPEST INSERTION')
    # print('Initial:')
    min_dist = None
    for origin in input_data.keys():
      for destin in input_data.keys():
        if origin != destin:
          new_dist = dist[origin][destin]
          # print(' distance between ' + origin + ' and ' + destin + ': ' + str(new_dist))

          if min_dist is None or new_dist < min_dist:
            min_dist = new_dist
            tour = [origin, destin, origin]

    while len(tour) <= len(input_data.keys()):
      print('Tour: ' + str(tour) + ', dist=' + str(sum(dist[tour[pos]][tour[pos+1]] for pos in range(len(tour)-1))) + r" \\")

      next_tour = None
      min_detour = None

      for node in input_data.keys():
        if node not in tour:
          print('--Insert ' + node + r" \\")
          for pos in range(len(tour)-1):
            left = tour[pos]
            right = tour[pos+1]
            c_left = dist[left][node]
            c_right = dist[node][right]
            c_init = dist[left][right]
            detour = c_left + c_right - c_init
            print('---between ' + left + ' and ' + right + ': detour=' + str(c_left) + '+' + str(c_right) + '-' + str(c_init) + '=' + str(detour) + r" \\")

            if next_tour is None or min_detour > detour:
              # print(tour[:pos+1])
              # print(tour[pos+1:])
              next_tour = tour[:pos+1] + [node] + tour[pos+1:]
              # print(next_tour)
              min_detour = detour
              # print(next_tour)

      tour = next_tour
      print(r" \\")


    # Info für Ergebnis-Tabelle
    total_distance = sum(dist[tour[pos]][tour[pos+1]] for pos in range(len(tour)-1))

    print('Tour: ' + str(tour))
    print('Total distance: ' + str(total_distance))

    tour_info['heuristik'].append(heuristik)
    tour_info['start'].append('n/a')
    tour_info['distance'].append(total_distance)
    tour_info['tour'].append(tour)


# Dataframe für Plot hinzufügen
data_map = {'name': [], 'id': [], 'lon': [], 'lat': [], 'start': []}
for i in range(len(tour_info['heuristik'])):
  for n in tour_info['tour'][i]:
    data_map['name'].append(n)
    data_map['id'].append(input_data[n]['id'])
    data_map['lon'].append(input_data[n]['lon'])
    data_map['lat'].append(input_data[n]['lat'])
    if tour_info['heuristik'][i] == 'Nächster Nachbar':
      data_map['start'].append('NN [' + str(input_data[tour_info['start'][i]]['id']) + ': ' + str(tour_info['start'][i]) + '], dist: ' + str(tour_info['distance'][i]) + ' km')
    else:
      data_map['start'].append('Cheapest Insertion, dist: ' + str(tour_info['distance'][i]) + ' km')


# Tabelle
df_tab = pd.DataFrame(data=tour_info)
# print(df_tab[['heuristik', 'start', 'distance']])
print(df_tab[['heuristik', 'start', 'distance']].to_latex(index=False))

# Plot
df_map = pd.DataFrame(data=data_map)
# print(df_map)
fig = px.line_geo(df_map, lon='lon', lat='lat', text='id',
                  fitbounds='locations',
                  hover_name='name',
                  facet_col='start',
                  facet_col_wrap=2,
                  height=4*200,
                  width=2*400
                  )
fig.update_traces(
    marker=dict(size=10),
    textposition='bottom center' # ['top left', 'top center', 'top right', 'middle left', 'middle center', 'middle right', 'bottom left', 'bottom center', 'bottom right']
)
fig.show()


NÄCHSTER NACHBAR
Startknoten: Halle (Saale) \\
  Tour: ['Halle (Saale)', 'Weißenfels', 'Dessau-Roßlau', 'Lutherstadt Wittenberg', 'Magdeburg', 'Halberstadt', 'Stendal', 'Halle (Saale)'] \\
  Distance: 473 = sum([33, 74, 27, 73, 47, 96, 123]) \\~\\
Startknoten: Magdeburg \\
  Tour: ['Magdeburg', 'Halberstadt', 'Halle (Saale)', 'Weißenfels', 'Dessau-Roßlau', 'Lutherstadt Wittenberg', 'Stendal', 'Magdeburg'] \\
  Distance: 409 = sum([47, 76, 33, 74, 27, 97, 55]) \\~\\
Startknoten: Dessau-Roßlau \\
  Tour: ['Dessau-Roßlau', 'Lutherstadt Wittenberg', 'Halle (Saale)', 'Weißenfels', 'Halberstadt', 'Magdeburg', 'Stendal', 'Dessau-Roßlau'] \\
  Distance: 411 = sum([27, 62, 33, 99, 47, 55, 88]) \\~\\
Startknoten: Lutherstadt Wittenberg \\
  Tour: ['Lutherstadt Wittenberg', 'Dessau-Roßlau', 'Halle (Saale)', 'Weißenfels', 'Halberstadt', 'Magdeburg', 'Stendal', 'Lutherstadt Wittenberg'] \\
  Distance: 400 = sum([27, 42, 33, 99, 47, 55, 97]) \\~\\
Startknoten: Weißenfels \\
  Tour: ['Weißenfels', '

<ipython-input-40-4cfdfab9179d>:111: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.

